In [85]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import re
from nltk.corpus import stopwords
from pymystem3 import Mystem
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from nltk import FreqDist
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import  ward, dendrogram
from gensim import corpora, models
import gensim
from gensim.models.doc2vec import *
from sklearn.pipeline import Pipeline

In [86]:
import pyLDAvis.gensim
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

In [87]:
df = pd.read_csv('subdata.csv', usecols = ['title', 'text', 'topic', 'tags'])
df.dropna(subset=['tags', 'topic'], inplace = True)
df.shape

(10000, 4)

In [88]:
df.head(10)

,title,text,topic,tags
0,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика
1,Полет российских бомбардировщиков над Карибски...,Минобороны опубликовало на YouTube видео полет...,Мир,Политика
2,Британцам придется заплатить за путешествия по...,Жители Великобритании будут платить семь евро ...,Мир,Политика
3,Бутину допросят в США по новому уголовному делу,"Россиянка Мария Бутина, судимая в США по обвин...",Мир,Политика
4,Пентагон проигнорировал обращение России по ра...,Пентагон проигнорировал обращение министра обо...,Мир,Политика
5,Испанские клубы открестились от Неймара,Нападающий «Пари Сен-Жермен» Неймар принял реш...,Спорт,Футбол
6,Россия предложила Китаю участвовать в судьбе р...,Россия призвала Китай принять участие в перего...,Мир,Политика
7,Моуринью сравнил футболистов с мебелью,Главный тренер «Манчестер Юнайтед» Жозе Моурин...,Спорт,Футбол
8,Путин подарил тяжелобольному мальчику обещанны...,Президент России Владимир Путин организовал дл...,Россия,Политика
9,"Путин порассуждал о сексе, наркотиках и протес...","Президент России Владимир Путин рассказал, что...",Россия,Политика


In [89]:
df['topic'].value_counts()

Мир                2911
Спорт              2296
Интернет и СМИ     1631
Россия             1623
Культура           1028
Наука и техника     511
Name: topic, dtype: int64

In [90]:
df['tags'].value_counts()

Политика    4534
Футбол      2296
Интернет    1631
Кино        1028
Космос       511
Name: tags, dtype: int64

Оставим только тексты с 5 тегами:

In [91]:
tags = ['Космос', 'Политика', 'Кино', 'Футбол', 'Интернет']
df = df.loc[df['tags'].isin(tags)]
df.shape

(10000, 4)

__Проведем стандартную предобработку текста:__

In [92]:
regex = re.compile("[А-Яа-я]+")
def words_only(text, regex=regex):
    return " ".join(regex.findall(text))


mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д', 'с', 'год', 'в', 'на', 'м']
def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.lower().split() if not token in mystopwords])
    except:
        return ""

    
m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return [l for l in m.lemmatize(text) if l.isalpha()]
    except:
        return []

def preprocessing(text):
    words = words_only(text)
    no_stopwords = remove_stopwords(words)
    lemmas = lemmatize(no_stopwords)
    return(lemmas)

In [93]:
s = np.array(df.text)[0]
print(s)
preprocessing(s)

С начала расследования российского вмешательства в выборы власти США потратили более 25 миллионов долларов. Об этом сообщает Associated Press со ссылкой на отчет Министерства юстиции США. В документе содержатся данные о расходах на следствие с апреля по сентябрь 2018 года. За эти полгода было потрачено 4,6 миллиона долларов, из которых почти 3 миллиона долларов ушли на зарплату сотрудников, 580 тысяч — на поездки и сопутствующие расходы. Ранее Минюст США уже публиковал отчеты о затратах на дело о российском вмешательстве за предыдущие месяцы. 11 декабря расследование спецпрокурора Робера Мюллера показало, что по меньшей мере 14 человек из окружения президента США Дональда Трампа контактировали с россиянами во время его избирательной кампании и последующего переходного периода перед вступлением в должность главы государства. Мюллер с 2017 года ведет дело о якобы российском вмешательстве в американские выборы в 2016-м. Перед ним поставлена задача выяснить, был ли сговор между штабом Трам

['начинать',
 'расследование',
 'российский',
 'вмешательство',
 'выборы',
 'власть',
 'сша',
 'потратить',
 'миллион',
 'доллар',
 'сообщать',
 'ссылка',
 'отчет',
 'министерство',
 'юстиция',
 'сша',
 'документ',
 'содержаться',
 'данные',
 'расход',
 'следствие',
 'апрель',
 'сентябрь',
 'год',
 'полгода',
 'потратить',
 'миллион',
 'доллар',
 'который',
 'миллион',
 'доллар',
 'уходить',
 'зарплата',
 'сотрудник',
 'тысяча',
 'поездка',
 'сопутствующий',
 'расход',
 'ранее',
 'минюст',
 'сша',
 'публиковать',
 'отчет',
 'затрата',
 'дело',
 'российский',
 'вмешательство',
 'предыдущий',
 'месяц',
 'декабрь',
 'расследование',
 'спецпрокурор',
 'робер',
 'мюллер',
 'показывать',
 'меньший',
 'мера',
 'человек',
 'окружение',
 'президент',
 'сша',
 'дональд',
 'трамп',
 'контактировать',
 'россиянин',
 'время',
 'избирательный',
 'кампания',
 'последующий',
 'переходный',
 'период',
 'вступление',
 'должность',
 'глава',
 'государство',
 'мюллер',
 'год',
 'вести',
 'дело',
 'якобы',

In [94]:
%time df['lemmas'] = df.text.apply(lambda x: preprocessing(x))

Wall time: 27min 20s


In [95]:
df.head()

,title,text,topic,tags,lemmas
0,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика,"[начинать, расследование, российский, вмешател..."
1,Полет российских бомбардировщиков над Карибски...,Минобороны опубликовало на YouTube видео полет...,Мир,Политика,"[минобороны, опубликовывать, видео, полет, рос..."
2,Британцам придется заплатить за путешествия по...,Жители Великобритании будут платить семь евро ...,Мир,Политика,"[житель, великобритания, быть, платить, семь, ..."
3,Бутину допросят в США по новому уголовному делу,"Россиянка Мария Бутина, судимая в США по обвин...",Мир,Политика,"[россиянка, мария, бутин, судить, сша, обвинен..."
4,Пентагон проигнорировал обращение России по ра...,Пентагон проигнорировал обращение министра обо...,Мир,Политика,"[пентагон, проигнорировать, обращение, министр..."


__Тематическое моделирование__

In [96]:
dictionary = corpora.Dictionary(df.lemmas)
corpus = [dictionary.doc2bow(text) for text in df.lemmas]

In [97]:
%time ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=20)

Wall time: 26.8 s


In [98]:
ldamodel.print_topics(num_topics=5, num_words=4)

[(0, '0.014*"матч" + 0.013*"клуб" + 0.012*"команда" + 0.010*"футболист"'),
 (1, '0.010*"который" + 0.009*"год" + 0.006*"сообщать" + 0.006*"свой"'),
 (2, '0.015*"год" + 0.013*"фильм" + 0.008*"актер" + 0.006*"который"'),
 (3,
  '0.018*"россия" + 0.012*"российский" + 0.011*"президент" + 0.011*"украина"'),
 (4, '0.011*"сша" + 0.009*"год" + 0.008*"россия" + 0.007*"который"')]

In [99]:
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [105]:
%%time

model = Word2Vec(df.lemmas, size=100, window=3, min_count=20, workers=4)

Wall time: 482 ms


In [106]:
model.save("word2v.model")
model = Word2Vec.load("word2v.model")

In [107]:
model['новость']

array([ 0.44603863,  0.37664676,  0.12182311, -0.1692134 , -0.00685446,
        1.0283936 ,  0.10235171, -0.38360962, -0.3975512 , -0.15950964,
        0.31206867,  1.348694  ,  0.27198228, -0.56158376, -0.4228304 ,
        0.03067837,  0.277327  , -0.08198454,  0.03988322, -0.91072255,
       -0.8027857 ,  0.5109663 , -0.2624125 ,  0.00474986,  0.90951914,
        0.14493965,  0.30554834,  0.02169066,  0.34756678, -0.96176785,
       -0.79623497, -0.00325411, -0.04767684, -0.04631065, -0.03997865,
       -0.47120252, -0.26427752,  0.7884972 , -0.12740383,  0.5289211 ,
        0.53070223, -0.02364918,  0.6492594 , -0.6757414 ,  0.8047822 ,
       -0.26187697,  0.762255  ,  0.07969748,  0.07408977,  0.04125132,
       -1.5373418 ,  0.15125747, -0.24913949,  0.7880057 , -0.04232217,
       -0.13694972,  0.08733755,  0.24484603,  0.68905264,  0.38688165,
        0.04735612,  0.27852097, -0.10360334, -0.35184208, -0.46208268,
        0.02226362, -1.0168408 ,  0.46728697, -0.9590706 ,  0.03

In [108]:
model.most_similar("сайт")

[('страница', 0.9923110008239746),
 ('автор', 0.9895975589752197),
 ('никто', 0.9891266226768494),
 ('интернет', 0.9888849258422852),
 ('публикация', 0.988490641117096),
 ('социальный', 0.9882526993751526),
 ('актер', 0.9881114959716797),
 ('обращать', 0.9879593849182129),
 ('письмо', 0.9876288175582886),
 ('скриншот', 0.9871417284011841)]

__Визуализируем полученное пространство векторов:__

In [109]:
top_words = []

fd = FreqDist()
for text in tqdm(df.lemmas):
    fd.update(text)
for i in fd.most_common(1000):
    top_words.append(i[0])
print(top_words[:15])


['год', 'который', 'россия', 'российский', 'сообщать', 'президент', 'сша', 'заявлять', 'страна', 'ноябрь', 'слово', 'свой', 'становиться', 'время', 'отмечать']


In [110]:
top_words = [w for w in top_words if len(w) > 4]
len(top_words)

867

In [111]:
top_words_vec = model[top_words]

__t-SNE__

In [112]:
tsne = TSNE(n_components=2, random_state=0)

%time top_words_tsne = tsne.fit_transform(top_words_vec)

Wall time: 5.77 s


In [113]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

__PCA__

In [114]:
pca = PCA(n_components=2)
%time pca_words = pca.fit_transform(top_words_vec)

Wall time: 160 ms


In [115]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec PCA for most common words")

source = ColumnDataSource(data=dict(x1=pca_words[:,0],
                                    x2=pca_words[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)